# 2.2 内置序列类型概览
容器序列
    list、tuple、collections.deque 这些序列能存放不同类型的数据
扁平序列
    str、bytes、bytearray、memoryview 和 array.array，这类序列只能容纳一种类型

按照可变性分类
    可变序列
        list、bytearray、array.array、collections.deque 和 memoryview
    不可变序列
        tuple、str 和 bytes

# 2.3 列表推导和生成器表达式
  ## 2.3.1 列表推导式对可读性的影响

In [ ]:
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
print(codes)

codes = [ord(symbol) for symbol in symbols]
print(codes)

如果你不打算使用生成的列表，那就不要使用列表推导式语法。
  ## 2.3.2 列表推导式与 filter 和 map 的比较

In [ ]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
print(beyond_ascii)

beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
print(beyond_ascii)

## 2.3.3 笛卡尔积

In [ ]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors
                        for size in sizes]
print(tshirts)

for color in colors:
    for size in sizes:
        print((color, size))

## 2.3.4 生成器表达式
    虽然列表推导式也可以生成元组、数组或其他序列类型，但是生成器表达式占用的内存更少。

In [ ]:
symbols = '$¢£¥€¤'
print(tuple(ord(symbol) for symbol in symbols))

import array
print(array.array('I', (ord(symbol) for symbol in symbols)))

生成器表达式一次产出一项。

In [ ]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in (f'{c} {s}' for c in colors for s in sizes):
    print(tshirt)

# 2.4 元组不仅仅是不可变列表
    元组有两个作用，除了可以作为不可变列表使用之外，还可用作没有字段名称的记录。
## 2.4.1 用作记录
    如果只把元组当作不可变列表，那么项数和项的顺序就不重要了。
    如果把元组当作字段的容器使用，那么项数通常是固定的，顺序也变得十分重要。

In [ ]:
lax_coordinates = (33.9425, -118.408056)
# 元组拆包
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'),
                ('ESP', 'XDA205856')]
for passport in sorted(traveler_ids):
    # 元组拆包
    print('%s/%s' % passport)

for country, _ in traveler_ids:
    print(country)

## 2.4.2 用作不可变列表
    Python解释器和标准库经常把元组当作不可变列表来使用。
    意图清晰
        只要在源码中见到元组，你就知道它的长度不可变。
    性能优越
        长度相同的元组和列表，元组占用的内存更少，而且Python可对元组做些优化。

    元组的不可变性仅针对元组中的引用，如果引用的是可变对象且引用对象发生了变化，元组中的值也会变化。

In [9]:
a = (10, 'alpha', [1,2])
b = (10, 'alpha', [1,2])
print(a == b)
b[-1].append(3)
print(a == b)

True
False


只有值永不可变的对象才是可哈希的。不可哈希的元组不能作用字典的键，也不能作为集合的元素。

In [ ]:
# 显式判断一个元组（或其他对象）的值是否固定
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True

tf = (10, 'alpha', (1,2))
print(fixed(tf))
tm = (10, 'alpha', [1,2])
print(fixed(tm))

元组性能上的优势
    · Python编译器求解元组字面量时，一次操作即可生成元组常量的字节码。求解列表字面量时，生成的字节码将是每个元素当作独立的常量推入数据栈，然后构建列表。
    · 给定一个元组t, tuple(t)直接返回t的引用。给定一个列表l,list(l)创建l的副本。
    · tuple实例长度固定，分配的内存空间正好够用。而list实例的内存空间要富余一些，时刻准备追加元素。
    · 对元组中项的引用存储在元组结构体内的一个数组中，而列表把引用数组的指针存储在别处。原因是列表可以变长，这会导致CPU缓存效率较低。

## 2.4.3 列表和元组方法的比较
    元组支持所有不涉及增删改的列表方法，元组没有__reversed__方法，但是可以使用内置函数reversed， reversed(my_tuple)。

# 2.5 序列和可迭代对象拆包
    拆包的目标可以是任何可迭代对象，包括不支持索引表示法（[]）的迭代器。
    最明显的拆包是并行赋值（parallel assigment）

In [ ]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates
print(latitude)
print(longitude)

print(divmod(20, 8))
t = (20, 8)
print(*t)
print(divmod(*t))
quotient, remainder = divmod(*t)
print(quotient)
print(remainder)

## 2.5.1 使用*获取余下的项
    定义函数时可以使用*args捕获余下的任意数量的参数。

In [ ]:
a, b, *rest = range(5)
print(a, b, rest)
a, b, *rest = range(3)
print(a, b, rest)
a, b, *rest = range(2)
print(a, b, rest)

# 并行赋值时，*前缀只能用在一个变量名前面，但这个变量可以出现在赋值表达式的任意位置

a, *body, c, d = range(5)
print(a, body, c, d)
*head, b, c, d = range(5)
print(head, b, c, d)

## 2.5.2 在函数调用和序列字面量中使用*拆包
    在函数调用中可以多次使用*。

In [ ]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest

print(fun(*[1, 2], 3, *range(4,7)))

# 定义列表、元组或集合字面量时，也可以使用*。
print(*range(4), 4)

print([*range(4), 4])

print({*range(4), 4, *(5, 6, 7)})


## 2.5.3 嵌套拆包
    拆包的对象可以嵌套，要保证值的嵌套结构相同。

In [ ]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
]

def main():
    # ：15 表示宽度为15个字符，
    # >9 表示数值右对齐且总宽度为9，
    # 9.4f 表示将浮点数格式化为宽度为 9 个字符，其中包括小数点和小数部分的 4 位。左侧的空白会用空格填充。
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for name, _, _, (latitude, longitude) in metro_areas:
        if longitude <= 0:
            print(f'{name:15} | {latitude:9.4f} | {longitude:9.4f}')

if __name__ == '__main__':
    main()

拆包赋值的对象也可以是一个列表，不过用途不大。例如一个数据库查询只返回一个记录，利用拆包确保只返回一个结果。
\[record\] = query_returning_one_record()

## 2.6 序列模式匹配
    Python3.10中引入了match/case语句实现的模式匹配。
    https://peps.python.org/pep-0634/

In [ ]:
def handle_command(self, message):
    match message:
        case ['BEEPER', frequency, times]:
            self.beep(times, frequency)
        case ['NECK', angle]:
            self.rotate_neck(angle)
        case ['LED', ident, intensity]:
            self.leds[ident].set_brightness(ident, intensity)
        case ['LED', ident, red, green, blue]:
            self.leds[ident].set_color(ident, red, green, blue)
        case _:
            raise InvalidCommand(message)

match/case看上去很像C的switch/case，但不同点在于match支持析构。

In [ ]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
]

def main():
    # ：15 表示宽度为15个字符，
    # >9 表示数值右对齐且总宽度为9，
    # 9.4f 表示将浮点数格式化为宽度为 9 个字符，其中包括小数点和小数部分的 4 位。左侧的空白会用空格填充。
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            # case语句由两部分组成，第一部分是模式，第二部分是使用if关键字指定的卫语句（guard clause）。可选
            case [name, _, _, (latitude, longitude)] if longitude <= 0:
                print(f'{name:15} | {latitude:9.4f} | {longitude:9.4f}')


一般来说，匹配对象需要同时满足以下条件才能匹配序列模式
    · 匹配对象是序列
    · 匹配对象的长度与模式长度相同
    · 匹配对象的每个元素都与模式的对应元素匹配
序列模式可以匹配collections.abc.Sequence的多数实际子类或者虚拟子类的实力，但是str, bytes, bytearray除外。

标准库中的以下类型与序列模式兼容。
    · list
    · tuple
    · range
    · collections.deque
    · array.array
    · memoryview

_符号在模式中有特殊意义：匹配相应位置上的任何一项但是不绑定匹配项的值。另外，_是唯一可在模式中多次出现的变量。

添加类型信息可以让模式更具体。
case \[str(name), _, _, (float(lat), float(lon))\]:
在模式上下文中str(name)和float(lat)都是类型检查，而不是构造函数调用。
可以用*_匹配任意数量的项且不绑定变量。
case \[str(name), *_, (float(lat), float(lon))\]:
以if开头的卫语句是可选的，仅当匹配模式时才会执行。

# 2.7 切片
## 2.7.1 为什么切片和区间排除最后一项
    Python的切片和区间都是左闭右开的，这样做的优点有：
    · 当只有最后一个位置信息时，可以快速看出切片和区间里有几个元素：range(3)和my_list\[:3\]都返回3个元素。
    · 当起止位置信息都可见时，可以快速计算出切片和区间的长度，用后一个数减去第一个下标（stop - start）即可。
    · 可以利用任意一个下标把序列分割成不重叠的两部分，只要写成my_list\[:x\]和my_list\[x:\]就可以了。


In [18]:
l = [1, 2, 3]
print(l[:2])
print(l[2:])

[1, 2]
[3]


## 2.7.2 切片对象
    我们可以使用s[a:b:c]的形式对s在a和b之间以c为间隔取值。c的值还可以为负数，负数意味着反向取值。

In [ ]:
s = 'bicycle'
print(s[::3])
print(s[::-1])
print(s[::-2])

a:b:c这种用法只能作为索引或下标用在[]中来返回一个切片对象：slice(a, b, c)。可以单独为切片对象命名。

In [ ]:
invoice = """
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                    $17.50    3    $52.50
1489  6mm Tactile Switch x20                $4.95    2     $9.90
1510  Panavise Jr. - PV-201                $28.00    1    $28.00
1601  PiTFT Mini Kit 320x240               $34.95    1    $34.95
"""
SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55,None)
line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

## 2.7.3 多维切片和省略号
    []运算符还可以接受多个索引或切片，用逗号分隔。例如，a\[m:n, k:l\]返回一个二维切片对象，它包含a的第m到n-1行、第k到l-1列的交叉区域。
    除了memoryview之外，没有内置的类型支持多维切片。
    省略号写作(...), 它其实是Ellipsis的别名，Ellipsis是Python中的单例，可以用a\[...\]取代a\[:\]。

## 2.7.4 给切片赋值
    

In [ ]:
l = list(range(10))
print(l)
l[2:5] = [20, 30]
print(l)
del l[5:7]
print(l)
l[3::2] = [11, 22]
print(l)
try:
    l[2:5] = 100
except TypeError:
    print('Type Error')
# 如果赋值目标是一个切片，则右边必须是一个可迭代对象，即使只有一个值。
l[2:5] = [100]
print(l)

# 2.8 使用+和*处理序列
    通常+两端的两个对象必须为同一类型的序列，结果是一个同类型的新序列。
    如果想多次拼接同一个序列，可以乘一个整数。

In [ ]:
l = [1, 2, 3]
print(l * 5)
print(5 * 'abcd')
# 注意乘法中的运算对象中包含可变项的话，复制出来的是多个引用，而不是副本

## 2.8.1 构建嵌套列表

In [ ]:
board = [['_'] * 3 for i in range(3)]
print(board)
board[1][2] = 'X'
print(board)

# 等同于
board = []
for i in range(3):
    row = ['_'] * 3
    board.append(row)

# 错误的示例
weird_board = [['_'] * 3] * 3
print(weird_board)
weird_board[1][2] = 'O'
print(weird_board)

# 等同于
row = ['_'] * 3
board = []
for i in range(3):
    board.append(row)

## 2.8.2 使用增量赋值运算符处理序列
    +=背后的特殊方法是\_\_iadd\_\_（就地相加），但是如果一个类没有实现这个方法的话，Python会退一步调用\_\_add\_\_。

In [ ]:
l = [1, 2, 3]
print(id(l))
l *= 2
print(l)
# l没有变
print(id(l))

t = (1, 2, 3)
print(id(t))
t *= 2
# 创建了一个新元组t
print(t)

## 2.8.3 一个+=运算符赋值谜题

In [ ]:
t = (1, 2, [30, 40])
try:
    t[2] += [50, 60]
except TypeError as e:
    print(e)
print(t)

s\[a\] += b的字节码解释
1. 把s\[a\]的值放在栈顶（TOS）。
2. 执行 TOS += b。如果TOS引用的是一个可变对象，则操作成功。
3. 赋值s\[a\] = TOS。如果s是不可变对象，则操作失败。

教训
1. 不要把可变对象放在元组里面。
2. 增量赋值不是一个原子操作。我们刚才也看到了，它虽然抛出了异常，但还是完成了操作。
3. 查看Python的字节码并不难，而且它对我们了解代码背后的运行机制很有帮助。

# 2.9 list.sort与内置函数sorted
    list.sort方法就地排序列表，返回值为None。
    内置函数sorted会新建一个列表作为返回值。

    list.sort和sorted均接受两个可选的关键字参数。
    reverse
        为True时降序返回项，默认False
    key
        一个只接受一个参数的函数，应用到每一项上，作为排序依据。

In [30]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
print(sorted(fruits))
print(fruits)
print(sorted(fruits, reverse=True))
print(sorted(fruits, key=len))
print(sorted(fruits, key=len, reverse=True))
print(fruits)
fruits.sort()
print(fruits)

['apple', 'banana', 'grape', 'raspberry']
['grape', 'raspberry', 'apple', 'banana']
['raspberry', 'grape', 'banana', 'apple']
['grape', 'apple', 'banana', 'raspberry']
['raspberry', 'banana', 'grape', 'apple']
['grape', 'raspberry', 'apple', 'banana']
['apple', 'banana', 'grape', 'raspberry']


# 2.10 当列表不适用时
    使用数组处理上百万个浮点值可以节省大量内存。
    如果经常需要在列表两端添加或者删除项，使用deque更合适。
## 2.10.1 数组
    如果一个列表只包含数值，那么使用array.array会更高效。

In [ ]:
from array import array
from random import random
floats = array('d', (random() for i in range(10**7)))
print(floats[-1])

## 2.10.2 memoryview
    内置的memoryview类是一种共享内存的序列类型，可在不复制字节的情况下处理数组的切片。

In [ ]:
from array import array
octets = array('B', range(6))
m1 = memoryview(octets)
print(m1.tolist())

m2 = m1.cast('B', [2, 3])
print(m2.tolist())

m3 = m1.cast('B', [3, 2])
print(m3.tolist())

m2[1, 1] = 22
m3[1, 1] = 33
print(octets)


numbers = array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
print(len(memv))
print(memv[0])
memv_oct = memv.cast('B')
print(memv_oct.tolist())
memv_oct[5] = 4
print(numbers)


## 2.10.3 NumPy
    NumPy实现了多维同质数组和矩阵，这些数据结构不但能处理数字，还能存放其他由用户定义的记录。
    在NumPy基础上编写的SciPy库提供了大量的数值算法，例如求解线性代数、数值积分和统计学上常用的算法。

In [39]:
!pip3 install numpy

import numpy as np
a = np.arange(12)
print(a)
print(type(a))
print(a.shape)
a.shape = 3, 4
print(a)
print(a[2])
print(a[2, 1])
print(a[:, 1])
print(a.transpose())

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/55/78/f85aab3bda3ddffe6ce8c590190b5f0d2e61dfd2fb7a8f446dcb4f8c12c7/numpy-1.26.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 183.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 19.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
[ 0  1  2  3  4  5  6  7  8  9 10 11]
<class 'numpy.ndarray'>
(12,)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[ 8  9 10 11]
9
[1 5 9]
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]


## 2.10.4 双端队列和其他队列
    collections.deque类（双端队列）是一个线程安全、可以快速从两端添加或者删除元素的数据类型。

In [45]:
from collections import deque
dq = deque(range(10), maxlen=10)
print(dq)
dq.rotate(3)
print(dq)
dq.rotate(-4)
print(dq)
dq.appendleft(-1)
print(dq)
dq.extend([11, 22, 33])
print(dq)
dq.extendleft([10, 20, 30, 40])
print(dq)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33], maxlen=10)
deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8], maxlen=10)
